# Onboard DEXPI data to CDF

**Prerequisite**:

- Installed Neat, see [Installation](../../../gettingstarted/installation.html)
- Launched a notebook environment.
- Familiar with the `NeatSession` object, see [introduction](../../introduction/introduction.html)
- Access to `NeatEngine`.

In this tutorial, we will:
- extract a knowledge graph from a `dexpi` P&ID file
- transform knowledge graph for further usage in CDF
- infer a data model based on the transformed knowledge graph
- and finally push both the data model and instances from knowledge graph to CDF


Read more about dexpi here: https://www.dexpi.org/

## Extract knowledge graph from dexpi P&ID file

We will start by instantiating a `NeatSession` and read the data from a local folder.
Make sure you have a dexpi P&ID file in the folder. In case of this tutorial we are using a DEXPI file provided by DISC initiative. You can download file from <a href="https://github.com/cognitedata/neat/raw/refs/heads/main/tests/data/depxi_example.xml" download="depxi_example.xml">this link</a> (if download does not start right click and save link as).

```python

In [1]:
from cognite.neat import NeatSession, get_cognite_client

In [2]:
# Note that we use Oxigraph in this example, this will not work in a CDF notebook
# to make it work in a CDF notebook replace set storage="memory"

neat = NeatSession(get_cognite_client(".env"), storage="oxigraph")

Found .env file in repository root. Loaded variables from .env file.
Neat Engine 2.0.3 loaded.


Process of extracting graph from a dexpi file can take a bit of time, so be patient.

In [3]:
neat.read.xml.dexpi("./data.xml")

If we now check content of the `graph` object, we will see that it contains a knowledge graph extracted from the dexpi file:

In [4]:
# This cell is removed from the docs.
from cognite.neat._config import GLOBAL_CONFIG

# Switch to TQDM progress bar as that looks better in the docs
GLOBAL_CONFIG.progress_bar = "tqdm-notebook"

In [5]:
neat

,Type,Occurrence
0,GenericAttribute,770
1,Association,330
2,PipingNetworkSegment,29
3,Nozzle,23
4,PipingNetworkSystem,20
...,...,...
42,PipeReducer,1


Now we will tranform this graph by calling dexpi specific bundle of transformers, which will:
- attach values of generic attributes to nodes
- create associations between nodes
- remove unused generic attributes
- remove associations between nodes that do not exist in the extracted graph
- remove edges to nodes that do not exist in the extracted graph

In [6]:
neat.prepare.instances.dexpi()

If we run notebook locally we can preview a part of knowledge graph which should have `ProcessPlant` in its center:

In [6]:
neat.show.instances()

instances.html


If we now inspect content of neat session we can notice that we went from 1306 to 206 instances, since we perfomed all the above transformations in order to produce rich and well connected graph.
Observ that number of types went only from 43 to 41, and that is because we move information from `GenericAttribute` and `Association` to correct nodes, making new properties and edges.
These two types where then consecutively removed as all their information has been used.

In [7]:
neat

,Type,Occurrence
0,PipingNetworkSegment,29
1,Nozzle,23
2,PipingNetworkSystem,20
3,Flange,19
4,ProcessInstrumentationFunction,19
...,...,...
40,PipeReducer,1


Studying the output above, we see that we succesfully read the assets and activities.

## Infer Data Model

We can infer a data model from data in the `NeatSession` by calling `.infer()`.

In [8]:
neat.infer()

Inferring classes:   0%|          | 0/41 [00:00<?, ?it/s]

[]

In [9]:
neat

This gives us an unverified data model, which we can then verify

## Verify Data Model

In [10]:
neat.verify()

[]

In [11]:
neat

We now convert verified data model to be DMS compliant

In [12]:
neat.convert("dms")

Rules converted to dms


[]

## Inspect Data Model

First we will set data model id to ("dexpi_playground", "DEXPI", "v1.3.1"), where tuple represents (space, external_id, version) of data model

We use the `.show` to inspect the data model and see how data model objects are connected, the resulting interactive data model visualization should be similar to this one:


In [13]:
neat.show.data_model()

http_purl.org_cognite_neat_data-model_verified_physical_neat_space_NeatInferredDataModel_v1.html


In [14]:
neat.set.data_model_id(("dexpi_playground", "DEXPI", "v1.3.1"))

[]

In [15]:
neat

## Publishing Data Model to CDF

Now we are ready to publish this to CDF.

In [30]:
neat.to.cdf.data_model()

You can inspect the details with the .inspect.outcome.data_model(...) method.


,name,created
0,schema,0
1,spaces,1
2,containers,41
3,views,41
4,data_models,1
5,nodes,0


<img src="../../../artifacts/figs/dexpi-dm-cdf.png" width="1200">

## Populating Data Model

Neat keeps track of the data, so we can immidiately populate this data model with the original data

In [31]:
neat.to.cdf.instances()

INFO | 2024-12-09 12:04:54,228 | Staring DMSLoader and will process 41 views.
INFO | 2024-12-09 12:04:54,230 | Starting ViewId(space='dexpi_playground', external_id='ActuatingElectricalFunction', version='v1.3.1') 1/41.
INFO | 2024-12-09 12:04:54,248 | Finished ViewId(space='dexpi_playground', external_id='ActuatingElectricalFunction', version='v1.3.1').
INFO | 2024-12-09 12:04:54,249 | Starting ViewId(space='dexpi_playground', external_id='ActuatingFunction', version='v1.3.1') 2/41.
INFO | 2024-12-09 12:04:54,257 | Finished ViewId(space='dexpi_playground', external_id='ActuatingFunction', version='v1.3.1').
INFO | 2024-12-09 12:04:54,258 | Starting ViewId(space='dexpi_playground', external_id='ActuatingSystem', version='v1.3.1') 3/41.
INFO | 2024-12-09 12:04:54,265 | Finished ViewId(space='dexpi_playground', external_id='ActuatingSystem', version='v1.3.1').
INFO | 2024-12-09 12:04:54,265 | Starting ViewId(space='dexpi_playground', external_id='BallValve', version='v1.3.1') 4/41.
INFO 

You can inspect the details with the .inspect.outcome.instances(...) method.


,name,created
0,ActuatingElectricalFunction,1
1,ActuatingFunction,4
2,ActuatingSystem,4
3,BallValve,3
4,BlindFlange,1
5,ButterflyValve,1
6,CentrifugalPump,1
7,CheckValve,1
8,ControlledActuator,4
9,CustomHeatExchanger,1


<img src="../../../artifacts/figs/dexpi-instance-cdf.png" width="1200">